In [21]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = "http://spheresofpower.wikidot.com/alchemy"

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'slayer-talents/'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []



    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        
        return result

    def talent_info_search(self):
        all_data_dicts = []
        url_list = []

        spheres = ["Alchemy", "Athletics", "Barrage", "Barroom", "Beastmastery", "Berserker", "Boxing", "Brute", "Dual-Wielding", "Duelist", "Equipment", "Fencing", "Gladiator", "Guardian", "Lancer", "Open-Hand", "Scoundrel", "Scout", "Shield", "Sniper", "Trap", "Warleader", "Wrestling"]
        for sphere in spheres:
            url = f"http://spheresofpower.wikidot.com/{sphere}"   
            url_list.append(url)

        for url2 in url_list:
            self.detail_page = requests.get(url2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')
            div_tags = self.detail_soup.find_all('div', id="page-content")
            i = 0
            for div in div_tags:
                h_tags = div.find_all('h4')
                for h_tag in h_tags:
                    print(h_tag)
                    p_tags = div.find_all('p')
                    p_tag = p_tags[i]
                    print(p_tag)
                    data_dict = {}
                    data_dict[h_tag.get_text()] = {}
                    i += 1

                for p_tag in p_tags:
                    # Split text at ':' to get key-value pairs
                    p_tag_clean = self.remove_parenthesis(p_tag.get_text())
                    parts = p_tag_clean.split(':')
                    if len(parts) > 1:
                        key = self.remove_parenthesis(parts[0].strip())
                        key = self.clean_text(key)
                        value = self.remove_parenthesis(parts[1].strip())
                        value = self.remove_parenthesis(value)
                        data_dict[h_tag.get_text()][key] = value

                all_data_dicts.append(data_dict)

        # Convert the list of dictionaries to a JSON string
        json_data = json.dumps(all_data_dicts, indent=2)
        with open('spheres_of_power.json', 'w', encoding='utf-8') as json_file:
            json.dump(all_data_dicts, json_file, indent=2, ensure_ascii=False)        

        # Print or return the JSON string as needed
        print(json_data)
        return json_data



    def main(self):        
        # print(self.soup.prettify())
        self.talent_info_search()

        
 
# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)
scraper_instance.main()

<h4 id="toc4"><span>Billowing Poison</span></h4>
<p>When you gain the Alchemy sphere, you gain 5 ranks in the Craft (alchemy) skill, plus 5 ranks per additional talent spent in the Alchemy sphere (maximum ranks equal to your total Hit Dice). If you already have ranks in the Craft (alchemy) skill you may immediately retrain them, but you do not get to retrain when only temporarily gaining talents, such as through the armiger’s customized weapons class feature.</p>
<h4 id="toc5"><span>Careful Poisoner</span></h4>
<p>In addition, choose one of the following packages:</p>
<h4 id="toc6"><span>Chemical Coating [Apoc]</span></h4>
<p>Some Alchemy talents have the (formulae) descriptor; these are improved or original versions of alchemical items you can only create once you’ve learned the talent. All formulae have a base crafting DC as noted in their descriptor, and their saving throw DCs, if any, are determined using the normal formula for combat sphere talents, using the practitioner’s ranks 

In [5]:
spheres = ["Alchemy", "Athletics", "Barrage", "Barroom", "Beastmastery", "Berserker", "Boxing", "Brute", "Dual-Wielding", "Duelist", "Equipment", "Fencing", "Gladiator", "Guardian", "Lancer", "Open-Hand", "Scoundrel", "Scout", "Shield", "Sniper", "Trap", "Warleader", "Wrestling"]


In [6]:
for sphere in spheres:
    url = f"http://spheresofpower.wikidot.com/{sphere}"    
    print(url)

http://spheresofpower.wikidot.com/Alchemy
http://spheresofpower.wikidot.com/Athletics
http://spheresofpower.wikidot.com/Barrage
http://spheresofpower.wikidot.com/Barroom
http://spheresofpower.wikidot.com/Beastmastery
http://spheresofpower.wikidot.com/Berserker
http://spheresofpower.wikidot.com/Boxing
http://spheresofpower.wikidot.com/Brute
http://spheresofpower.wikidot.com/Dual-Wielding
http://spheresofpower.wikidot.com/Duelist
http://spheresofpower.wikidot.com/Equipment
http://spheresofpower.wikidot.com/Fencing
http://spheresofpower.wikidot.com/Gladiator
http://spheresofpower.wikidot.com/Guardian
http://spheresofpower.wikidot.com/Lancer
http://spheresofpower.wikidot.com/Open-Hand
http://spheresofpower.wikidot.com/Scoundrel
http://spheresofpower.wikidot.com/Scout
http://spheresofpower.wikidot.com/Shield
http://spheresofpower.wikidot.com/Sniper
http://spheresofpower.wikidot.com/Trap
http://spheresofpower.wikidot.com/Warleader
http://spheresofpower.wikidot.com/Wrestling
